In [42]:
import os
import glob
import shutil
import pandas as pd
from astropy.io import fits
from sunpy.io.header import FileHeader

In [43]:
harpnum_dir = "allharp2noaa2.csv"
main_dir = "/data/SHARPS/raw-sharps"
destination_folder = "/data/SHARP_HARP"

In [44]:
folder_list = os.listdir(main_dir)
df = pd.read_csv(harpnum_dir)
harpnum_list = df["DEF_HARPNUM"].tolist()

In [45]:
def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))

def listdir_nohidden(path):
    return glob.glob(os.path.join(path, '*'))

In [72]:
common_list = intersection(folder_list, list(map(str, harpnum_list)))
common_folder_list = [os.path.join(main_dir, i) for i in common_list]
for folder in common_folder_list:
    files_inside_folder = listdir_nohidden(folder)
    magnetogram_files = [file for file in files_inside_folder if 'magnetogram' in file]

    for magnetogram_file in magnetogram_files:
        time = magnetogram_file.split("_")[3]
        if time in ['000000', '120000']:
            
            # for getting latlong 
            hdul = fits.open(magnetogram_file)
            hdr = hdul[1].header 
            LAT_MIN, LON_MIN, LAT_MAX, LON_MAX = hd['LAT_MIN'], hd['LON_MIN'], hd['LAT_MAX'], hd['LON_MAX']
            if LAT_MIN<70 and LON_MIN>70 and LAT_MAX<80 and LON_MAX>80:
                
                #saving
                source = os.path.join(main_dir, folder, magnetogram_file)
                destination = os.path.join(destination_folder, os.mkdir(folder))
                shutil.copy(source, destination)

time 001200
time 051200
time 192400
time 072400
time 012400
time 010000
time 070000
time 183600
time 033600
time 063600
time 190000
time 224800
time 144800
time 114800
time 030000
time 060000
time 134800
time 023600
time 180000
time 043600
time 204800
time 154800
time 041200
time 021200
time 002400
time 052400
time 033600
time 144800
time 051200
time 192400
time 012400
time 021200
time 002400
time 060000
time 134800
time 180000
time 043600
time 204800
time 060000
time 023600
time 154800
time 041200
time 182400
time 002400
time 031200
time 012400
time 070000
time 190000
time 124800
time 214800
time 053600
time 041200
time 011200
time 182400
time 062400
time 002400
time 000000
/data/SHARPS/raw-sharps/5163/hmi.sharp_cea_720s.5163.20150206_000000_TAI.magnetogram.fits hdul [<astropy.io.fits.hdu.image.PrimaryHDU object at 0x7fc651844100>, <astropy.io.fits.hdu.compressed.CompImageHDU object at 0x7fc6518446a0>] 

time 193600
time 060000
time 234800
time 180000
time 073600
time 023600
time 1048

time 210000
time 140000
time 203600
time 014800
time 044800
time 163600
time 133600
time 231200
time 151200
time 101200
time 142400
time 212400
time 122400
time 171200
time 221200
time 111200
time 152400
time 102400
time 091200
time 232400
time 093600
time 160000
time 130000
time 184800
time 200000
time 004800
time 113600
time 080000
time 223600
time 173600
time 064800
time 130000
time 093600
time 150000
time 200000
time 054800
time 004800
time 213600
time 123600
time 173600
time 221200
time 111200
time 141200
time 202400
time 152400
time 132400
time 231200
time 161200
time 101200
time 112400
time 142400
time 222400
time 081200
time 194800
time 120000
/data/SHARPS/raw-sharps/5163/hmi.sharp_cea_720s.5163.20150213_120000_TAI.magnetogram.fits hdul [<astropy.io.fits.hdu.image.PrimaryHDU object at 0x7fc99815de20>, <astropy.io.fits.hdu.compressed.CompImageHDU object at 0x7fc6515c51f0>] 

time 170000
time 083600
time 210000
time 103600
time 014800
time 074800
time 163600
time 233600
time 0900

time 160000
time 213600
time 054800
time 080000
time 123600
time 141200
time 102400
time 232400
time 112400
time 222400
time 170000
time 044800
time 090000
time 141200
time 192400
time 072400
time 010000
time 213600
time 054800
time 123600
time 170000
time 134800
time 090000
time 204800
time 112400
time 222400
time 182400
time 151200
time 062400
time 000000
/data/SHARPS/raw-sharps/6857/hmi.sharp_cea_720s.6857.20161119_000000_TAI.magnetogram.fits hdul [<astropy.io.fits.hdu.image.PrimaryHDU object at 0x7fc6515ffe80>, <astropy.io.fits.hdu.compressed.CompImageHDU object at 0x7fc651696100>] 

time 044800
time 203600
time 133600
time 160000
time 124800
time 214800
time 080000
time 102400
time 232400
time 170000
time 044800
time 203600
time 133600
time 090000
time 151200
time 112400
time 222400
time 102400
time 232400
time 160000
time 054800
time 080000
time 171200
time 091200
time 042400
time 184800
time 020000
time 113600
time 223600
time 064800
time 140000
time 193600
time 234800
time 2124

time 230000
time 100000
time 033600
time 144800
time 180000
time 153600
time 024800
time 131200
time 201200
time 230000
time 100000
time 143600
time 034800
time 082400
time 121200
time 211200
time 162400
time 092400
time 131200
time 201200
time 110000
time 153600
time 024800
time 071200
time 142400
time 191200
time 120000
/data/SHARPS/raw-sharps/6857/hmi.sharp_cea_720s.6857.20161121_120000_TAI.magnetogram.fits hdul [<astropy.io.fits.hdu.image.PrimaryHDU object at 0x7fc6516965e0>, <astropy.io.fits.hdu.compressed.CompImageHDU object at 0x7fc651696be0>] 

time 210000
time 164800
time 013600
time 093600
time 004800
time 173600
time 221200
time 111200
time 083600
time 120000
/data/SHARPS/raw-sharps/6857/hmi.sharp_cea_720s.6857.20161123_120000_TAI.magnetogram.fits hdul [<astropy.io.fits.hdu.image.PrimaryHDU object at 0x7fc6512ae370>, <astropy.io.fits.hdu.compressed.CompImageHDU object at 0x7fc651212fa0>] 

time 210000
time 014800
time 163600
time 231200
time 101200
time 142400
time 221200
ti

time 072400
time 021200
time 134800
time 204800
time 043600
time 062400
time 022400
time 182400
time 021200
time 134800
time 040000
time 043600
time 000000
/data/SHARPS/raw-sharps/3323/hmi.sharp_cea_720s.3323.20131103_000000_TAI.magnetogram.fits hdul [<astropy.io.fits.hdu.image.PrimaryHDU object at 0x7fc652345520>, <astropy.io.fits.hdu.compressed.CompImageHDU object at 0x7fc6512ae370>] 

time 124800
time 214800
time 084800
time 053600
time 164800
time 013600
time 071200
time 191200
time 031200
time 072400
time 032400
time 031200
time 192400
time 050000
time 124800
time 053600
time 010000
time 134800
time 043600
time 094800
time 204800
time 174800
time 003600
time 061200
time 181200
time 021200
time 000000
/data/SHARPS/raw-sharps/3323/hmi.sharp_cea_720s.3323.20131029_000000_TAI.magnetogram.fits hdul [<astropy.io.fits.hdu.image.PrimaryHDU object at 0x7fc651235640>, <astropy.io.fits.hdu.compressed.CompImageHDU object at 0x7fc65194a0a0>] 

time 193600
time 010000
time 072400
time 182400
ti

...

time 084800
time 142400
time 131200
time 201200
time 120000
/data/SHARPS/raw-sharps/7085/hmi.sharp_cea_720s.7085.20170715_120000_TAI.magnetogram.fits hdul [<astropy.io.fits.hdu.image.PrimaryHDU object at 0x7fc6515c3550>, <astropy.io.fits.hdu.compressed.CompImageHDU object at 0x7fc6510aa5b0>] 

time 210000
time 142400
time 221200
time 111200
time 173600
time 093600
time 142400
time 002400
time 131200
time 201200
time 153600
time 024800
time 120000
/data/SHARPS/raw-sharps/7085/hmi.sharp_cea_720s.7085.20170717_120000_TAI.magnetogram.fits hdul [<astropy.io.fits.hdu.image.PrimaryHDU object at 0x7fc65194a220>, <astropy.io.fits.hdu.compressed.CompImageHDU object at 0x7fc6515c3220>] 

time 210000
time 060000
time 004800
time 173600
time 230000
time 093600
time 100000
time 221200
time 082400
time 111200
time 152400
time 012400
time 121200
time 211200
time 143600
time 034800
time 130000
time 070000
time 200000
time 014800
time 163600
time 083600
time 220000
time 110000
time 092400
time 231200
ti

/data/SHARPS/raw-sharps/6302/hmi.sharp_cea_720s.6302.20160129_000000_TAI.magnetogram.fits hdul [<astropy.io.fits.hdu.image.PrimaryHDU object at 0x7fc652345610>, <astropy.io.fits.hdu.compressed.CompImageHDU object at 0x7fc652345490>] 

time 201200
time 214800
time 053600
time 162400
time 124800
time 020000
time 121200
time 082400
time 211200
time 143600
time 042400
time 034800
time 031200
time 230000
time 100000
time 104800
time 172400
time 234800
time 073600
time 131200
time 193600
time 050000
time 153600
time 024800
time 032400
time 220000
time 110000
time 042400
time 181200
time 061200
time 003600
time 174800
time 020000
time 094800
time 082400
time 000000
/data/SHARPS/raw-sharps/6302/hmi.sharp_cea_720s.6302.20160131_000000_TAI.magnetogram.fits hdul [<astropy.io.fits.hdu.image.PrimaryHDU object at 0x7fc6515c5d60>, <astropy.io.fits.hdu.compressed.CompImageHDU object at 0x7fc6512e5d60>] 

time 161200
time 122400
time 023600
time 212400
time 154800
time 041200
time 140000
time 182400
ti

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



[<astropy.io.fits.hdu.image.PrimaryHDU object at 0x7fc6510a21c0>, <astropy.io.fits.hdu.compressed.CompImageHDU object at 0x7fc6510a2130>] 

time 162400
time 141200
time 082400
time 183600
time 123600
time 054800
time 213600
time 100000
time 230000
time 024800
time 090000
time 153600
time 052400
time 011200
time 170000
time 073600
time 222400
time 112400
time 104800
time 201200
time 030000
time 131200
time 172400
time 151200
time 193600
time 092400
time 133600
time 203600
time 044800
time 110000
time 220000
time 080000
time 034800
time 042400
time 143600
time 001200
time 160000
time 232400
time 063600
time 114800
time 102400
time 211200
time 121200
time 020000
time 163600
time 052400
time 014800
time 110000
time 011200
time 083600
time 220000
time 073600
time 172400
time 234800
time 104800
time 101200
time 030000
time 193600
time 092400
time 231200
time 224800
time 063600
time 152400
time 114800
time 211200
time 121200
time 020000
time 183600
time 034800
time 042400
time 143600
time 001

TypeError: object of type 'NoneType' has no len()